In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv("f1.csv")

# Select features (you can add more)
features = [
    'SST_MIN', 'SST_MAX', 'SST@90th_HS', 'SSTA@90th_HS',
    'DHW_from_90th_HS>1', 'Salinity', 'pH_total', 'fCO2', 'IOD', 'ENSO'
]
target = 'Bleaching'

# Normalize features
scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

# Sort by reef and time
df = df.sort_values(by=['Reef_Andaman & Nicobar Islands', 'Year', 'Month'])

# Group by reef and create sequences
X, y = [], []
sequence_length = 12

grouped = df.groupby('Reef_Andaman & Nicobar Islands')
for _, group in grouped:
    group = group.reset_index(drop=True)
    for i in range(len(group) - sequence_length):
        X.append(group[features].iloc[i:i+sequence_length].values)
        y.append(group[target].iloc[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, len(features))))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Get predicted probabilities
y_pred_prob = model.predict(X_test).flatten()

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, digits=4))

# Plot confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
disp.plot(cmap="Blues", values_format='d')
plt.title("Confusion Matrix")
plt.show()


ModuleNotFoundError: No module named 'pandas'